# Collision Division - Investigating the data

In [1]:
import pandas as pd
import time
import math as m
import csv

In [2]:
# The first 10 lines were removed from the log file
data = pd.read_csv("20181108-201421_RTH80_input.log", header=None)

In [3]:
# Previewing the data
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.541708e+09,callsign --------,latitude 38.879638 longitude 282.379883,altitude 10965.180000 metres x,groundspeed 189.128838 m/s,airspeed ---- m/s,heading 218.927544,icao a6b911,airbourne,cat 0,nic 7,ADS-B,current,baro,NaN
1,1.541708e+09,callsign --------,latitude 38.868118 longitude 282.482260,altitude 9144.000000 metres x,groundspeed ---- m/s,airspeed ---- m/s,heading ----,icao a894f2,airbourne,cat 0,nic 8,ADS-B,current,baro,NaN
2,1.541708e+09,callsign --------,latitude 38.879638 longitude 282.379883,altitude 10965.180000 metres x,groundspeed 189.128838 m/s,airspeed ---- m/s,heading 218.927544,icao a6b911,airbourne,cat 0,nic 7,ADS-B,current,baro,NaN
3,1.541708e+09,callsign --------,latitude 38.868118 longitude 282.482260,altitude 9144.000000 metres x,groundspeed 193.256586 m/s,airspeed ---- m/s,heading 230.292070,icao a894f2,airbourne,cat 0,nic 8,ADS-B,current,baro,NaN
4,1.541708e+09,callsign --------,latitude 38.879638 longitude 282.379883,altitude 10965.180000 metres x,groundspeed 189.128838 m/s,airspeed ---- m/s,heading 218.927544,icao a6b911,airbourne,cat 0,nic 7,ADS-B,current,baro,NaN


In [4]:
# The number of rows and columns of the data
data.shape

(1129928, 15)

In [5]:
# Checking the column values for the 12th column
data[12].value_counts()

current    982735
stale      147193
Name: 12, dtype: int64

In [6]:
# Setting the column headers
col_names = ['timestamp', 'callsign', 'lat/long', 'altitude_(m)', 'groundspeed_(m/s)',
            'airspeed_(m/s)', 'heading', 'icao', 'status', 'cat', 'nic', 
             'transponder_type', 'current/stale', 'baro', 'unknown']

data.columns = col_names

In [7]:
# Previewing the data
data.head(1)

,timestamp,callsign,lat/long,altitude_(m),groundspeed_(m/s),airspeed_(m/s),heading,icao,status,cat,nic,transponder_type,current/stale,baro,unknown
0,1.541708e+09,callsign --------,latitude 38.879638 longitude 282.379883,altitude 10965.180000 metres x,groundspeed 189.128838 m/s,airspeed ---- m/s,heading 218.927544,icao a6b911,airbourne,cat 0,nic 7,ADS-B,current,baro,NaN


In [8]:
# Checking the values of the unknown column
data['unknown'].value_counts()

Series([], Name: unknown, dtype: int64)

In [9]:
# Dropping the last column (only has NaN values)
data.drop('unknown', axis=1, inplace=True)

In [10]:
data.head(1)

,timestamp,callsign,lat/long,altitude_(m),groundspeed_(m/s),airspeed_(m/s),heading,icao,status,cat,nic,transponder_type,current/stale,baro
0,1.541708e+09,callsign --------,latitude 38.879638 longitude 282.379883,altitude 10965.180000 metres x,groundspeed 189.128838 m/s,airspeed ---- m/s,heading 218.927544,icao a6b911,airbourne,cat 0,nic 7,ADS-B,current,baro


## Cleaning up the columns

### Removing the word 'callsign' from every value

In [11]:
# Checking the callsign values
data.callsign.value_counts()

callsign --------    72395
callsign 00000000    14653
callsign DAL1241     13031
callsign N858CV      12909
callsign N6712J      12509
callsign NKS146      12494
callsign DAL1561     12399
callsign N500RE      12254
callsign JBU719      11996
callsign DAL1992     11682
callsign VIV3945     11391
callsign N45RR       11358
callsign SKW3625     11323
callsign JBU165      11289
callsign UAL1286     11084
callsign JBU119      11077
callsign SWA296      10944
callsign N544CM      10862
callsign AAL731      10789
callsign N591SF      10580
callsign SWA2218     10536
callsign DAL1544     10438
callsign AAL2032     10431
callsign DLH428      10368
callsign EDV3444     10314
callsign RPA4733     10115
callsign N512CZ      10097
callsign N690NG       9963
callsign FFT1139      9685
callsign DAL2162      9463
                     ...  
callsign DCM6323       783
callsign EJA555        724
callsign ENY3659       686
callsign N7254P        676
callsign N404MM        638
callsign DAL1570       622
c

In [12]:
# Removing the word 'callsign' from every row in callsign
data['callsign'] = data['callsign'].str.split().str[1]

In [13]:
# Replacing all '--------' values with a single - to make it easier
data['callsign'].replace('--------', '-', inplace=True)

In [14]:
data.callsign.value_counts()

-           72395
00000000    14653
DAL1241     13031
N858CV      12909
N6712J      12509
NKS146      12494
DAL1561     12399
N500RE      12254
JBU719      11996
DAL1992     11682
VIV3945     11391
N45RR       11358
SKW3625     11323
JBU165      11289
UAL1286     11084
JBU119      11077
SWA296      10944
N544CM      10862
AAL731      10789
N591SF      10580
SWA2218     10536
DAL1544     10438
AAL2032     10431
DLH428      10368
EDV3444     10314
RPA4733     10115
N512CZ      10097
N690NG       9963
FFT1139      9685
DAL2162      9463
            ...  
DCM6323       783
EJA555        724
ENY3659       686
N7254P        676
N404MM        638
DAL1570       622
RPA4584       606
N377SC        581
N1465E        533
N2220N        525
FFT1148       516
CLU815        513
GJS6289       513
SKW5236       507
AAL1506       497
RPA6044       486
AAL1955       473
ENY9801       463
N62DV         433
UAL752        397
RPA4580       394
AAL401        387
JBU6          382
SWA1767       376
N109DC    

In [15]:
# Dropping the callsign
data.drop('callsign', axis=1, inplace=True)

## Splitting out the latitude and longitude

In [16]:
# Splitting out the latitude and longitude
data['latitude'] = data['lat/long'].str.split().str[1]
data['longitude'] = data['lat/long'].str.split().str[3]

In [17]:
# Dropping the lat/long column
data.drop('lat/long', axis=1, inplace=True)

## Altitude column

In [18]:
# Splitting up the altitude column
data['altitude_(m)'][1].split()

['altitude', '9144.000000', 'metres', 'x']

In [19]:
# Checking if all units are metres (they are)
data['altitude_(m)'].str.split().str[2].value_counts()

metres    1129928
Name: altitude_(m), dtype: int64

In [20]:
# Keeping the x/+ value until I find out if it means anything
data['altitude_units_x/+'] = data['altitude_(m)'].str.split().str[3]

# Removing the word altitude from each value
data['altitude_(m)'] = data['altitude_(m)'].str.split().str[1]

In [21]:
# Replace '-----' with '-'
data['altitude_(m)'].replace('-----', '-', inplace=True)

In [22]:
# Three different units (X, +, -). Need to find out what they mean.
data['altitude_units_x/+'].value_counts()

x    827342
+    196091
-    106495
Name: altitude_units_x/+, dtype: int64

In [23]:
# Dropping the altitude units
data.drop('altitude_units_x/+', axis=1, inplace=True)

## Groundspeed column

In [24]:
# Inspecting the first value for groundspeed
data['groundspeed_(m/s)'][1].split()

['groundspeed', '----', 'm/s']

In [25]:
# Checking if the unit is always m/s
data['groundspeed_(m/s)'].str.split().str[2].value_counts()

m/s    1129928
Name: groundspeed_(m/s), dtype: int64

In [26]:
# Removing the word groundspeed and m/s
data['groundspeed_(m/s)'] = data['groundspeed_(m/s)'].str.split().str[1]

In [27]:
# Replacing '----' with '-'
data['groundspeed_(m/s)'].replace('----', '-', inplace=True)

## Airspeed column

In [28]:
# Inspecting the first value for airspeed
data['airspeed_(m/s)'][1].split()

['airspeed', '----', 'm/s']

In [29]:
# Checking the values of this column. Redundant.
data['airspeed_(m/s)'].value_counts()

airspeed ---- m/s    1129928
Name: airspeed_(m/s), dtype: int64

In [30]:
# Dropping the column from the dataframe
data.drop('airspeed_(m/s)', axis=1, inplace=True)

## Heading column

In [31]:
# Inspecting the first value for heading
data['heading'][1].split()

['heading', '----']

In [32]:
# Removing the word heading
data['heading'] = data['heading'].str.split().str[1]

In [33]:
# Replacing '----' with '-'
data['heading'].replace('----', '-', inplace=True)

## icao column

In [34]:
# Inspecting the first value for icao
data['icao'][1].split()

['icao', 'a894f2']

In [35]:
# Removing the word icao
data['icao'] = data['icao'].str.split().str[1]

## cat column

In [36]:
# Inspecting the first value for cat
data['cat'][1].split()

['cat', '0']

In [37]:
# Removing the word icao
data['cat'] = data['cat'].str.split().str[1]

In [38]:
# Checking the values of cat (not sure what cat is, but it's an int value of 0,1,2,4,5,6)
data['cat'].value_counts()

4    530840
0    255081
2    210955
1     72504
6     33885
5     26663
Name: cat, dtype: int64

In [39]:
# Dropping the cat column
data.drop('cat', axis=1, inplace=True)

## Nic column

In [40]:
# Inspecting the first value for nic
data['nic'][1].split()

['nic', '8']

In [41]:
# Removing the word icao
data['nic'] = data['nic'].str.split().str[1]

In [42]:
# Checking the values of cat (not sure what nic is, but it's an int value)
data['nic'].value_counts()

8     1020574
7       47865
9       25877
10      23751
11       5807
0        4097
12       1386
13        252
14        153
15         49
16         44
18         38
17         35
Name: nic, dtype: int64

In [43]:
# Dropping the nic column
data.drop('nic', axis=1, inplace=True)

## Checking values for other columns

In [44]:
# Checking the values for transponder type. All values are ADS-B
data['transponder_type'].value_counts()

ADS-B    1129928
Name: transponder_type, dtype: int64

In [45]:
# Dropping the transponder type column since data is redundant
data.drop('transponder_type', axis=1, inplace=True)

In [46]:
# Checking the values for current/stale. Binary.
data['current/stale'].value_counts()

current    982735
stale      147193
Name: current/stale, dtype: int64

In [47]:
# Dropping the current/stale column
data.drop('current/stale', axis=1, inplace=True)

In [48]:
# Checking the values for baro. Binary.
data['baro'].value_counts()

baro    1126656
geo        3272
Name: baro, dtype: int64

In [49]:
# Dropping the baro column
data.drop('baro', axis=1, inplace=True)

## Filtering the dataframe

In [50]:
# Filtering where the status is airbourne
data = data[data.status=='airbourne']

In [51]:
# Dropping the airbourne column
data.drop('status', axis=1, inplace=True)

In [52]:
# Filtering out the values where groundspeed is untracked
data = data[data['groundspeed_(m/s)']!='-']

## Rearranging the columns

In [53]:
data.columns

Index(['timestamp', 'altitude_(m)', 'groundspeed_(m/s)', 'heading', 'icao',
       'latitude', 'longitude'],
      dtype='object')

In [54]:
columns = ['timestamp', 'icao', 'latitude', 'longitude', 'altitude_(m)', 'groundspeed_(m/s)', 'heading']

data = data.reindex(columns=columns)

In [55]:
data.head()

,timestamp,icao,latitude,longitude,altitude_(m),groundspeed_(m/s),heading
0,1.541708e+09,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
2,1.541708e+09,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
3,1.541708e+09,a894f2,38.868118,282.482260,9144.000000,193.256586,230.292070
4,1.541708e+09,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
5,1.541708e+09,c07e46,38.711929,281.802421,11277.600000,250.767262,357.530870


In [56]:
# Converting the timestamp to integers
# data.timestamp  = data.timestamp.astype(int)

## Removing values where the icao value has the same lat/long

In [81]:
unique_data = data.groupby(['icao', 'latitude', 'longitude']).first().reset_index()

In [89]:
unique_data = unique_data.sort_values('timestamp').reset_index()

In [91]:
unique_data.drop('index', inplace=True, axis=1)

In [109]:
# Changing heading to a float
unique_data.heading = unique_data.heading.astype(float)

In [93]:
unique_data.columns1541708062

Index(['icao', 'latitude', 'longitude', 'timestamp', 'altitude_(m)',
       'groundspeed_(m/s)', 'heading'],
      dtype='object')

In [95]:
columns = ['icao', 'timestamp', 'longitude', 'latitude', 'altitude_(m)', 'heading', 'groundspeed_(m/s)']

unique_data = unique_data.reindex(columns=columns)

In [96]:
unique_data

,icao,timestamp,longitude,latitude,altitude_(m),heading,groundspeed_(m/s)
0,a6b911,1541708062,282.379883,38.879638,10965.180000,218.927544,189.128838
1,ac8d34,1541708062,282.577881,39.071200,9753.600000,219.008713,184.711348
2,c07e46,1541708062,281.802421,38.711929,11277.600000,357.530870,250.767262
3,a2925a,1541708062,282.504531,38.542419,6827.520000,201.072727,180.279567
4,ab38aa,1541708062,282.214827,38.723969,11582.400000,232.164704,195.413738
5,a894f2,1541708062,282.481245,38.867477,9144.000000,230.292070,193.256586
6,a894f2,1541708062,282.482260,38.868118,9144.000000,230.292070,193.256586
7,a894f2,1541708063,282.480349,38.866928,9144.000000,230.292070,193.256586
8,a2925a,1541708063,282.504173,38.541687,6835.140000,201.072727,180.279567
9,c07e46,1541708063,281.802246,38.715075,11277.600000,357.530870,250.767262


## Exporting the cleaned up csv file

In [98]:
data.head()

,timestamp,icao,latitude,longitude,altitude_(m),groundspeed_(m/s),heading
0,1541708062,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
2,1541708062,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
3,1541708062,a894f2,38.868118,282.482260,9144.000000,193.256586,230.292070
4,1541708062,a6b911,38.879638,282.379883,10965.180000,189.128838,218.927544
5,1541708062,c07e46,38.711929,281.802421,11277.600000,250.767262,357.530870


In [97]:
unique_data.to_csv("cleaned_data_collision_division.csv")

## Investigating the data

In [59]:
timedelta = data.timestamp.max()-data.timestamp.min()

# In hours
timedelta/60/60

42.840555555555554

In [60]:
data.timestamp.max(), data.timestamp.min()

(1541862288, 1541708062)

In [61]:
max_lat = data.latitude.max()
min_lat = data.latitude.min()

max_longitude = data.longitude.max()
min_longitude = data.longitude.min()

In [62]:
min_lat, max_lat

('37.447723', '39.968634')

In [63]:
min_longitude, max_longitude

('279.635726', '284.066932')